### Gpt API 간단 사용법 (그냥 Query만 넣어서) 정리

In [1]:
import os
import openai

# .env 파일에 openai api key 있음
openai.api_key = os.environ["OPENAI_API_KEY"]

query = """
"한 회사는 선불전자지급수단을 발행하고 있습니다. 분기당 발행잔액은 28억원, 연간 총발행액은 450억원입니다. 
이 경우, 이 회사는 선불업 등록을 해야 하나요?" 

위 질문을 의미적으로 문장 단위로 나눠주세요.
"""

response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": query},
    ],
)

print(response.choices[0].message.content)

1. "한 회사는 선불전자지급수단을 발행하고 있습니다. 
2. 분기당 발행잔액은 28억원, 연간 총발행액은 450억원입니다. 
3. 이 경우, 이 회사는 선불업 등록을 해야 하나요?"


### Retrieval augumented Generation - pdf에서 데이터를 가져옴

In [1]:
%pip install langchain
%pip install unstructured

  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.36-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
  Using cached aiohttp-3.11.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached langchain_core-0.3.19-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.1.143-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... \^C
anceled
Note: you may need to restart the kernel to use updated packages.
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached filetype-1.2.0-py

In [10]:
# 필요한 라이브러리 임포트
import os
import re
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import pdfplumber  # PDF 파일 로드에 사용하는 라이브러리

import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 클라이언트 및 Embedding 모델 설정
client = OpenAI(api_key=api_key)
embedding_model = OpenAIEmbeddings(openai_api_key=api_key)

pdf 하나만 Load 할 때

pdf 여러개 Load 할 때

txt 파일 Load 할 때
형식은 1.정보 2.정보 3.정보 - split은 저 기준으로 할 것.

In [11]:
import re

# 텍스트 파일 읽기
with open("data/txt/university_info.txt", "r", encoding="utf-8") as file:
    text = file.read()

# 번호 패턴을 기준으로 split
# sections = re.split(r'(?<=\d\.)\s+', text)  # 숫자. 뒤에 공백을 기준으로 split
sections = re.split(r'(?<=\d\.)', text)[1:]  # 숫자와 점 뒤에서 split, 첫 번째 빈 문자열 제거

# 첫 번째 섹션이 빈 문자열일 수 있으므로 제거
if sections[0] == "":
    sections = sections[1:]

# 결과 확인
for idx, section in enumerate(sections):
    print(f"\n[섹션 {idx+3}]\n{section[:100]}...")  # 각 섹션의 앞부분만 표시


[섹션 3]
개강날짜
인천대학교의 2024년도 2학기 개강일자는 2024년 9월 2일입니다.
2....

[섹션 4]
고집운영시간
인천대학교에 위치한 고집은 평일에만 운영, 오전 11시에 오픈해서 오후 7시에 마감합니다.
3....

[섹션 5]
학산도서관 운영시간
인천대학교의 학산도서관은 학기중에는 평일 기준 09시 ~ 21시, 주말 및 공휴일 기준 09시 ~ 17시입니다. 
방학 중에는 09시 ~ 17시입니다.(단, 중...

[섹션 6]
셔틀버스 운영시간
인천대학교의 등교셔틀버스는 평일 08시~10시에 수시로 운행하고,
하교셔틀버스는 평일 18시~19시에 운행합니다.
5....

[섹션 7]
수강신청날짜
인천대학교의 2024년도 2학기 수강신청은 8월19일에 시작됩니다.
6....

[섹션 8]
졸업학점
2020~2022학년도 입학생은 130~135학점 이상(공학계열:135~140이상) 이수해야합니다.
2023학년도 이후 입학생은 130학점 이상 이수해야합니다.
7....

[섹션 9]
종강날짜
인천대학교는 2024년도 12월 9일~13일동안 기말시험을 치르고 16일~20일이 보강주간입니다.
8....

[섹션 10]
기초교양
인천대학교의 2023학년도 이후 입학자는 글쓰기이론과실제 2학점, Academic English 2학점, 대학영어회화 2학점, 
컴퓨팅적사고와SW 2학점(해당학과), 대학...

[섹션 11]
인천대 음식점
인천대 안에 입점해있는 음식점에는 봉구스밥버거,포썸, 최고당돈가스, 샹차이, 토마토도시락, 고기굽는집, 그라찌에,
맘스터치, 샐러디,스낵바 등이 있습니다.
10....

[섹션 12]
체육 분과 동아리
인천대의 체육분과 동아리에는 효월검우회,PANG,산악부UIAC,INU W FC, BOSS, 돌핀, 바이킹, 다크호스, 퍼펙트, 싸우라비,
INU START W가 ...


In [12]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini-2024-07-18", openai_api_key=api_key)

# 임베딩 모델 설정
embeddings = OpenAIEmbeddings()
# 벡터 스토어 생성
vectorstore = Chroma.from_texts(sections, embeddings)
def search_and_generate_answer(query: str):
    # 벡터 스토어에서 쿼리와 유사한 정보를 검색
    results = vectorstore.similarity_search(query, k=1)
    
    context = results[0].page_content
    prompt = f"다음 정보에 기반하여 질문에 답해주세요, 만약 없다면 정보가 없다면 '정보가 없습니다.'라고 출력해줘 \n\n정보: {context}\n\n질문: {query}\n\n답변:"
    
    # LLM을 이용하여 답변 생성
    answer = llm(prompt)
    
    # 응답에서 content 부분만 추출하여 반환
    return answer.content    

In [13]:
# 예제 쿼리
query = "오늘의 날씨는 어때?"
print(search_and_generate_answer(query))

정보가 없습니다.


In [7]:
# 예제 쿼리
query = "고집은 언제 열어?"
print(search_and_generate_answer(query))

고집은 평일에 오전 11시에 열어.


In [8]:
# 예제 쿼리
query = "인천대학교 크기는 어때?"
print(search_and_generate_answer(query))

정보가 없습니다.


In [9]:
# 예제 쿼리
query = "인천대 2023학년도 졸업학점기준이 어떻게 돼?"
print(search_and_generate_answer(query))

정보가 없습니다.
